# Imports

In [1]:
import pandas as pd 
import numpy as np

### Base com atributos numéricos

In [2]:
# Base tirada de https://archive.ics.uci.edu/ml/datasets/Balance+Scale

balance = pd.read_csv('databases/balance-scale.csv',
                      header=None, 
                      names=["Pendência", "Peso_E", "Distância_E", "Peso_D", "Distância_D"])

# Substituindo a string de pendência para valores compreenciveis
balance = balance.replace({
    "B": "Balanceado", 
    "L": "Esquerda", 
    "R": "Direita", 
})

balance

,Pendência,Peso_E,Distância_E,Peso_D,Distância_D
0,Balanceado,1,1,1,1
1,Direita,1,1,1,2
2,Direita,1,1,1,3
3,Direita,1,1,1,4
4,Direita,1,1,1,5
...,...,...,...,...,...
620,Esquerda,5,5,5,1
621,Esquerda,5,5,5,2
622,Esquerda,5,5,5,3
623,Esquerda,5,5,5,4


In [3]:
balance.describe()

,Peso_E,Distância_E,Peso_D,Distância_D
count,625.000000,625.000000,625.000000,625.000000
mean,3.000000,3.000000,3.000000,3.000000
std,1.415346,1.415346,1.415346,1.415346
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000


In [4]:
balance.dtypes

Pendência      object
Peso_E          int64
Distância_E     int64
Peso_D          int64
Distância_D     int64
dtype: object

In [5]:
balance = pd.get_dummies(balance, 
                         columns=["Pendência"], 
                         prefix=["Classe"])
balance

,Peso_E,Distância_E,Peso_D,Distância_D,Classe_Balanceado,Classe_Direita,Classe_Esquerda
0,1,1,1,1,1,0,0
1,1,1,1,2,0,1,0
2,1,1,1,3,0,1,0
3,1,1,1,4,0,1,0
4,1,1,1,5,0,1,0
...,...,...,...,...,...,...,...
620,5,5,5,1,0,0,1
621,5,5,5,2,0,0,1
622,5,5,5,3,0,0,1
623,5,5,5,4,0,0,1
